<a href="https://colab.research.google.com/github/YasinEnigma/mci-chatbot/blob/main/task-007/part-03/tts_and_asr_telegram_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 217 kB 3.7 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.8.0-py3-none-any.whl size=200125 sha256=dfeaa30fddf391d50a759bcd47d6c800c29a0f2074d6dfcbb1e7f569171ce5f3
  Stored in directory: /root/.cache/pip/wheels/c2/05/70/8409792e663e67a70e057df1f18d070105c1c457b3f410bbb0
Successfully built pyTelegramBotAPI


In [3]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg libsox-fmt-mp3
!pip install unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.11.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=3ebd198f7acb790e098c9856ffaa39e66d1d6bd5e9ec402fdcd646fab0e34744
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libid3tag0 libmad0 libmagic-mgc libmagic1 libopencore-amrnb0
  libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  

'\nRemember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!\nAlternatively, you can uncomment the exit() below to crash and restart the kernel, in the case\nthat you want to use the "Run All Cells" (or similar) option.\n'

In [4]:
#downloading pretrained models:
!git clone https://github.com/YasinEnigma/mci-chatbot

Cloning into 'mci-chatbot'...
remote: Enumerating objects: 455, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 455 (delta 54), reused 39 (delta 9), pack-reused 309
Receiving objects: 100% (455/455), 324.49 MiB | 31.82 MiB/s, done.
Resolving deltas: 100% (146/146), done.
Checking out files: 100% (358/358), done.


In [5]:
import nemo
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.metrics.wer import word_error_rate
from nemo.utils import logging, exp_manager
import wave
import json
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read 


import soundfile as sf
import json
import IPython
import librosa
import telebot
import requests
import warnings
warnings.filterwarnings('ignore')

[NeMo W 2022-12-11 21:26:54 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


In [6]:
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(restore_path='mci-chatbot/task-003/asr_model.h')

[NeMo W 2022-12-11 21:26:58 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: manifests/fa/commonvoice_train_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - i
    - m
    - 'n'
    - p
    - r
    - s
    - t
    - 'y'
    - z
    - ا
    - ب
    - ت
    - ث
    - ج
    - ح
    - خ
    - د
    - ذ
    - ر
    - ز
    - س
    - ش
    - ص
    - ض
    - ط
    - ظ
    - ع
    - غ
    - ف
    - ق
    - ل
    - م
    - ن
    - ه
    - و
    - پ
    - چ
    - ژ
    - ک
    - گ
    - ی
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    num_workers: 8
    pin_memory: true
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[N

[NeMo I 2022-12-11 21:26:58 features:223] PADDING: 16


ERROR:numba.cuda.cudadrv.driver:Call to cuInit results in CUDA_ERROR_NO_DEVICE


[NeMo I 2022-12-11 21:26:59 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /content/mci-chatbot/task-003/asr_model.h.


In [7]:
# Load the Tacotron2Model
from nemo.collections.tts.models import Tacotron2Model
from nemo.collections.tts.models.base import SpectrogramGenerator, Vocoder

# Let's see what pretrained models are available
print(Tacotron2Model.list_available_models())

[PretrainedModelInfo(
	pretrained_model_name=tts_en_tacotron2,
	description=This model is trained on LJSpeech sampled at 22050Hz, and can be used to generate female English voices with an American accent.,
	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_tacotron2/versions/1.0.0/files/tts_en_tacotron2.nemo,
	class_=<class 'nemo.collections.tts.models.tacotron2.Tacotron2Model'>
)]


In [18]:
# We can load the pre-trained model as follows
tts_model = Tacotron2Model.from_pretrained("tts_en_tacotron2").eval().to('cpu')

[NeMo I 2022-12-11 21:32:13 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.11.0/tts_en_tacotron2/74d819609f47744063679b44e93a47f1/tts_en_tacotron2.nemo.
[NeMo I 2022-12-11 21:32:13 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.11.0/tts_en_tacotron2/74d819609f47744063679b44e93a47f1/tts_en_tacotron2.nemo
[NeMo I 2022-12-11 21:32:13 common:910] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-11 21:32:14 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.asr.data.audio_to_text.AudioToCharDataset
      manifest_filepath: /raid/LJSpeech/nvidia_ljspeech_train.json
      max_duration: null
      min_duration: 0.1
      trim: false
      int_values: false
      load_audio: true
      normalize: true
      sample_rate: 22050
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 48
      num_workers: 4
    
[NeMo W 2022-12-11 21:32:14 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.asr.data.audio_to_

[NeMo I 2022-12-11 21:32:14 features:223] PADDING: 16
[NeMo I 2022-12-11 21:32:15 save_restore_connector:243] Model Tacotron2Model was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0/tts_en_tacotron2/74d819609f47744063679b44e93a47f1/tts_en_tacotron2.nemo.


In [19]:
vocoder = Vocoder.from_pretrained("tts_hifigan").eval().to('cpu')

[NeMo I 2022-12-11 21:32:15 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.11.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-12-11 21:32:15 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.11.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-12-11 21:32:15 common:910] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-11 21:32:18 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-12-11 21:32:18 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-12-11 21:32:18 features:223] PADDING: 0


[NeMo W 2022-12-11 21:32:18 features:200] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-12-11 21:32:18 features:223] PADDING: 0
[NeMo I 2022-12-11 21:32:20 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [29]:
def tts_func(text):
  tokens = tts_model.parse(text)
  spectrogram = tts_model.generate_spectrogram(tokens = tokens)

  # Invert the spectrogram into audio samples
  audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
  audio = audio.cpu().detach().numpy()[0]
  return audio

In [20]:
API_TOKEN = ''

In [30]:
bot = telebot.TeleBot(API_TOKEN)

# Handle '/start' and '/help'
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    bot.reply_to(message, """\
Hi there, I am VoiceBot.
I am here to recieve your voices and return transcript back to you. Just say anything nice :)) \
""")

# Handle all other messages with content_type 'text' (content_types defaults to ['text'])
@bot.message_handler(func=lambda message: True)
def tts(message):
	text = message.text
	sf.write('message.wav', tts_func(text), 16000)
	audio = open('message.wav', 'rb')
	bot.send_audio(message.chat.id, audio)

@bot.message_handler(content_types=['voice'])
def asr(message):
	file_info = bot.get_file(message.voice.file_id)
	file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path))
	with open('voice.ogg','wb') as f:
		f.write(file.content)
	x,_ = librosa.load('voice.ogg', sr=16000)
	sf.write('voice.wav', x, 16000)
	bot.send_message(message.chat.id, 'File is Recieved!')
 
	text = asr_model.transcribe(paths2audio_files=['voice.wav'], batch_size=1)[0]
	bot.reply_to(message, f'Text is: \n {text}')

In [31]:
if __name__ == '__main__':
    bot.polling(none_stop=True)


bot.infinity_polling()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

2022-12-11 21:44:09,981 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
